In [162]:
import pandas as pd
import sys

sys.path.append("../")

from lib.uid import gen_uid
from lib.clean import clean_names, clean_races, clean_sexes

In [163]:
def read_personnel_and_employment_hist():
    dfa = pd.read_csv("../../../data/GA/2022-09-27/employment-history.csv")
    return dfa


In [164]:
dfa = read_personnel_and_employment_hist()
dfa

dfa.columns

Index(['Unnamed: 0', 'OKEY', ' NAME', ' AGENCY', ' RANK', ' STATUS',
       ' START DATE', ' END DATE '],
      dtype='object')

In [165]:


dfa = dfa.rename(columns={" NAME": "NAME", " START DATE": "hire_date", " END DATE ":"left_date", "OKEY": "OKEY_KEY"})

def split_names(df):
    names = df.NAME.str.lower().str.strip().str.extract(r"(\w+) (\w+) ?(.+)")

    df.loc[:, "last_name"] = names[0]
    df.loc[:, "middle_name"] = names[2]
    df.loc[:, "first_name"] = names[1]

    return df 

dfa = dfa.pipe(split_names)

dfa = dfa.pipe(gen_uid, ["first_name", "last_name", "middle_name"])
dfa




dfb = pd.read_csv("../../../data/GA/2022-09-27/officer_data-20231220144255.csv", encoding="latin1")
dfb = dfb.rename(columns={" SEX": "sex", " RACE ": "race", 
                              " YOB": "year_of_birth", " MIDDLE": "middle_name",
                              " LAST NAME": "last_name", " FIRST NAME": "first_name"})

dfb
    
dfb = dfb.pipe(clean_names, ["first_name", "last_name", "middle_name"])

    
dfb = dfb.pipe(gen_uid, ["first_name", "last_name", "middle_name"])
    
dfb = dfb[["year_of_birth", "race", "sex", "uid"]]

df = pd.merge(dfa, dfb, on="uid", how="left")

df["state"] = "GA"


df.sort_values("OKEY_KEY")

df.loc[:, "left_date"] = df.left_date.str.replace(r"0000-00-00", "", regex=True)

df = df[~((df.left_date.fillna("") == ""))]


df = df[df.duplicated(subset=["OKEY_KEY"], keep=False)]

df = df.rename(columns={" AGENCY": "agency"})

df = df.pipe(gen_uid, ["first_name", "middle_name", "last_name", "agency"])

events_df = df[["uid", "agency", "state", "hire_date"]]


# Unpivot the DataFrame
events_df = events_df.melt(id_vars=['uid', 'agency', 'state'], 
                  value_vars=['hire_date'], 
                  var_name='event_type', 
                  value_name='event_date')

# Generate event_uid using gen_uid function
events_df = events_df.pipe(gen_uid, ['uid', 'event_type', 'event_date'], "event_uid")

events_df.sort_values("uid")

events_df.to_csv("./../../../data/GA/match/events_georgia_dropped_na_left_date.csv", index=False)

In [ ]:
dup_uids_list = [x for x in events_df["uid"]]

per_df = df[["first_name", "middle_name", "last_name", "agency", "uid", "year_of_birth", "race", "sex", "state", "OKEY_KEY"]]

per_df = per_df[per_df.uid.isin(dup_uids_list)]

per_df.to_csv("./../../../data/GA/match/personnel_georgia_dropped_na_left_date.csv", index=False)